In [1]:
import openai
import os
from openai import OpenAI
import base64
import json
import time
import requests
from moderation_response import format_and_get_moderation_response, format_json_response
from moderation_classes import Categories, CategoryScores, Moderation


In [2]:
client = OpenAI()
api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
# Path to your image
image_path = "image_07.png"

# Extract the image name from image_path
image_name = os.path.basename(image_path)

# Create a text file with the same name as the image but with a .txt extension
output_file_name = os.path.splitext(image_name)[0] + ".txt"

# Full path to the output text file in the current working directory
output_file_path = os.path.join(os.getcwd(), output_file_name)

#prompt for the AI 
prompt="""Analyze the image provided, describing in detail its main elements. What are the primary subjects, objects, and setting? Are there any notable details or features in the image that stand out?

Examine and explain the color scheme of the image. Identify the dominant and accent colors. Discuss any contrasts and the potential meanings or symbolism associated with these colors. How do these colors interact within the image?

Evaluate the overall sentiment or mood conveyed by the image. Consider the influence of composition, color palette, subject matter, and artistic style. What emotions or messages does the image seem to express?"""

In [5]:
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [6]:
payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

In [7]:
# Record the start time
start_time = time.time()

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"API request took {elapsed_time:.2f} seconds")

API request took 2.03 seconds


In [8]:
formatted_response1 = format_json_response(response.json())

In [9]:
    # Parse the JSON response
    json_data = response.json()

    # Extract the content from the JSON response
    content = json_data["choices"][0]["message"]["content"]

    # Print the content
    print(content)

I'm sorry, I can't assist with that request.


In [46]:
content=("A hand gesture where one finger is raised, often interpreted as offensive in many societies")

response = client.moderations.create(input=content)

output = response.results[0]

In [48]:
print(output)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.00027851754566654563, harassment_threatening=7.53892095417541e-07, hate=1.5813642676221207e-05, hate_threatening=5.92774398455731e-08, self_harm=8.868826739671931e-08, self_harm_instructions=2.0566700342783406e-08, self_harm_intent=1.4280733751093067e-07, sexual=2.025187541221385e-06, sexual_minors=2.621816305725666e-11, violence=2.2854486815049313e-05, violence_graphic=6.82987462141682e-08, self-harm=8.868826739671931e-08, sexual/minors=2.621816305725666e-11, hate/threatening=5.92774398455731e-08, violence/graphic=6

In [49]:
formatted_response2=format_and_get_moderation_response(output)
print(formatted_response2)

Categories:
Harassment: False
Harassment Threatening: False
Hate: False
Hate Threatening: False
Self Harm: False
Self Harm Instructions: False
Self Harm Intent: False
Sexual: False
Sexual Minors: False
Violence: False
Violence Graphic: False
Category Scores:
Harassment Score: 0.00027851754566654563
Harassment Threatening Score: 7.53892095417541e-07
Hate Score: 1.5813642676221207e-05
Hate Threatening Score: 5.92774398455731e-08
Self Harm Score: 8.868826739671931e-08
Self Harm Instructions Score: 2.0566700342783406e-08
Self Harm Intent Score: 1.4280733751093067e-07
Sexual Score: 2.025187541221385e-06
Sexual Minors Score: 2.621816305725666e-11
Violence Score: 2.2854486815049313e-05
Violence Graphic Score: 6.82987462141682e-08
Flagged: False



In [ ]:
# Write both responses to the text file
with open(output_file_path, "w") as output_file:
    output_file.write(f"Image Name: {image_name}\n")
    output_file.write("\n\nImage Analysis gpt-4-vision-preview API:\n")
    output_file.write(formatted_response1)
    output_file.write("\n\nModeration Endpoint API:\n")
    output_file.write(formatted_response2)

# Print the path to the output file
print(f"Responses written to: {output_file_path}")